# Demo of _narrative graphs_


In [1]:
import kagglehub
import pandas as pd

path = kagglehub.dataset_download("rmisra/news-category-dataset", "News_Category_Dataset_v3.json")
data = pd.read_json(path, lines=True)
data.head()

,link,headline,category,short_description,authors,date
0,https://www.huffpost.com/entry/covid-boosters-...,Over 4 Million Americans Roll Up Sleeves For O...,U.S. NEWS,Health experts said it is too early to predict...,"Carla K. Johnson, AP",2022-09-23
1,https://www.huffpost.com/entry/american-airlin...,"American Airlines Flyer Charged, Banned For Li...",U.S. NEWS,He was subdued by passengers and crew when he ...,Mary Papenfuss,2022-09-23
2,https://www.huffpost.com/entry/funniest-tweets...,23 Of The Funniest Tweets About Cats And Dogs ...,COMEDY,"""Until you have a dog you don't understand wha...",Elyse Wanshel,2022-09-23
3,https://www.huffpost.com/entry/funniest-parent...,The Funniest Tweets From Parents This Week (Se...,PARENTING,"""Accidentally put grown-up toothpaste on my to...",Caroline Bologna,2022-09-23
4,https://www.huffpost.com/entry/amy-cooper-lose...,Woman Who Called Cops On Black Bird-Watcher Lo...,U.S. NEWS,Amy Cooper accused investment firm Franklin Te...,Nina Golgowski,2022-09-22


In [2]:
# create a sample
sample = data.sample(5000, random_state=42)
docs = sample["headline"] + "\n\n" + sample["short_description"]
categories = sample["category"]
timestamps = sample["date"]

In [3]:
from narrativegraph import NarrativeGraph

model = NarrativeGraph(
    sqlite_db_path="output/demo_db.sqlite",
    on_existing_db="overwrite"
)
model.fit(docs, categories=categories, timestamps=timestamps)

INFO:narrativegraph.pipeline:Adding 5000 documents to database
INFO:narrativegraph.pipeline:Extracting triplets
Extracting triplets: 100%|██████████| 5000/5000 [00:17<00:00, 288.78it/s]
INFO:narrativegraph.pipeline:Mapping entities and predicates
INFO:narrativegraph.pipeline:Mapping triplets


In [4]:
model.entities_

,id,label,frequency,doc_frequency,adjusted_tf_idf,first_occurrence,last_occurrence,alt_labels,category
0,1,a new cast member,1,1,0.000000,2021-09-28,2021-09-28,"[""a new cast member""]",[ENTERTAINMENT]
1,2,her hands,3,2,3333.333333,2013-03-06,2015-12-21,"[""her hand"",""her hands""]","[CRIME, STYLE & BEAUTY, STYLE & BEAUTY]"
2,3,7 Weird House Pets People,1,1,0.000000,2013-08-30,2013-08-30,"[""7 Weird House Pets People""]",[HOME & LIVING]
3,4,Richard Cameron,2,1,2500.000000,2012-05-10,2012-05-10,"[""Richard Cameron""]","[HOME & LIVING, HOME & LIVING]"
4,5,Safest Years,1,1,0.000000,2017-12-28,2017-12-28,"[""Safest Years""]",[POLITICS]
...,...,...,...,...,...,...,...,...,...
13677,13678,Fraudulent Unemployment Benefits Payments,1,1,0.000000,2013-04-28,2013-04-28,"[""Fraudulent Unemployment Benefits Payments""]",[BUSINESS]
13678,13679,Effective Treaty,1,1,0.000000,2012-12-04,2012-12-04,"[""Effective Treaty""]",[WELLNESS]
13679,13680,lavish meals,1,1,0.000000,2014-04-15,2014-04-15,"[""lavish meals""]",[TRAVEL]
13680,13681,The album,1,1,0.000000,2015-03-24,2015-03-24,"[""The album""]",[ENTERTAINMENT]


In [5]:
from narrativegraph.dto.filter import GraphFilter
for comm in model.find_communities(
    graph_filter=GraphFilter(minimum_node_frequency=2, maximum_node_frequency=10)
):
    if len(comm.members) > 1:
        print(comm)
        print()

members=[EntityLabel(id=2, label='her hands'), EntityLabel(id=1367, label='Ms. Watts'), EntityLabel(id=10607, label='her legs')] score=0.6666666666666666 density=0.6666666666666666 avg_pmi=3.4070323032825875 conductance=0.0

members=[EntityLabel(id=21, label='rich, hearty soups'), EntityLabel(id=5086, label='delectable curries')] score=1.0 density=1.0 avg_pmi=3.7336385601702595 conductance=0.0

members=[EntityLabel(id=451, label='This Dad'), EntityLabel(id=6021, label='the art'), EntityLabel(id=1736, label='Tens of Thousands of Dollars'), EntityLabel(id=8522, label='Emmanuel Macron'), EntityLabel(id=3533, label='The Importance'), EntityLabel(id=4750, label='the store'), EntityLabel(id=10382, label='contemporary opera'), EntityLabel(id=209, label='the shirts'), EntityLabel(id=7121, label='Her Suit'), EntityLabel(id=10132, label='the Russian leader'), EntityLabel(id=23, label='A Mom'), EntityLabel(id=2583, label='slingshot-wielding boys'), EntityLabel(id=3293, label='The Happiest Man'), 

In [6]:
# create server to be viewed in own browser which blocks execution of other cells
model.serve_visualizer()

INFO:     Started server process [76673]
INFO:     Waiting for application startup.
INFO:root:Database engine provided to state before startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)
INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [76673]
INFO:root:Server stopped by user


In [8]:
# create server to be viewed in this notebook which therefore runs in the background
server = model.serve_visualizer(block=False)

INFO:root:Server started in background on port 8001
INFO:     Started server process [37625]
INFO:     Waiting for application startup.
INFO:root:Database engine provided to state before startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8001 (Press CTRL+C to quit)


INFO:     127.0.0.1:52795 - "GET / HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:52795 - "GET /vis HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:52795 - "GET /vis/ HTTP/1.1" 304 Not Modified
INFO:     127.0.0.1:52796 - "GET /vis/static/css/main.66685047.css HTTP/1.1" 200 OK
INFO:     127.0.0.1:52795 - "GET /vis/static/js/main.7e003561.js HTTP/1.1" 200 OK
INFO:     127.0.0.1:52803 - "GET /graph/bounds HTTP/1.1" 200 OK
INFO:     127.0.0.1:52795 - "GET /favicon.ico HTTP/1.1" 404 Not Found
INFO:     127.0.0.1:52803 - "OPTIONS /graph HTTP/1.1" 200 OK
INFO:     127.0.0.1:52803 - "POST /graph HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:52803 - "OPTIONS /graph/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52803 - "POST /graph/ HTTP/1.1" 200 OK
INFO:     127.0.0.1:52803 - "GET /entities/1919 HTTP/1.1" 200 OK
INFO:     127.0.0.1:52803 - "GET /entities/1919/docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:53615 - "POST /graph HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:53615 - "P

In [7]:
server.show_iframe()

In [8]:
server.stop()

INFO:root:Background server stopped
